In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [2]:
df_data = pd.read_csv('../data/titanic/train.csv')

In [6]:
corr_matrix = df_data.corr()

In [7]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [24]:
pd.DataFrame(df_data.corrwith(df_data['Survived'])).rename(columns={0:'corlinear'}).sort_values('corlinear', ascending=False)

,corlinear
Survived,1.000000
Fare,0.257307
Parch,0.081629
PassengerId,-0.005007
SibSp,-0.035322
Age,-0.077221
Pclass,-0.338481


In [10]:
to_drop = [col for col in upper.columns if any(upper[col].abs() > 0.4)]

In [11]:
to_drop

['Parch', 'Fare']

In [14]:
corr_features = list(upper.index[upper['Parch'].abs() > 0.4])

In [15]:
corr_features

['SibSp']

In [18]:
corr_values = list(upper['Parch'][upper['Parch'].abs() > 0.4])


In [19]:
corr_values

[0.41483769862015263]

In [25]:
import xgboost  as xgb

In [45]:
xgb_params = {
    'booster': 'gbtree',
    'learning_rate': 0.05,
    'max_depth': 5,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'objective': 'reg:linear',
    'n_estimators': 1000,
    'min_child_weight': 3,
    'gamma': 0,
    'silent': True,
    'n_jobs': 4,
    'random_state': 2019,
    'verbose': 1
}
lgb_params = {
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'min_child_samples': 46,
    'min_child_weight': 0.01,
    'subsample_freq': 2,
    'num_leaves': 40,
    'max_depth': 7,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'verbose': -1,
    'seed': 2019
}

In [28]:
data = pd.DataFrame(data={"x":[1,2,3,4,5,3,4,5,6,7], "y":[0,1,0,1,1,1,0,0,1,0],"z":[1,2,3,3,2,1,3,2,2,1]})

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data[['x','z']], data['y'], test_size=0.2)

In [40]:
data['y'].unique().shape[0]

(2,)

In [37]:
model = xgb.XGBClassifier(**xgb_params, early_stopping=50,)

In [42]:
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)],eval_metric='logloss',verbose=-1, early_stopping_rounds=50)

[0]	validation_0-logloss:0.693186	validation_1-logloss:0.693225
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.693372	validation_1-logloss:0.693499
[2]	validation_0-logloss:0.690853	validation_1-logloss:0.693724
[3]	validation_0-logloss:0.691204	validation_1-logloss:0.694262
[4]	validation_0-logloss:0.687381	validation_1-logloss:0.710905
[5]	validation_0-logloss:0.687317	validation_1-logloss:0.710822
[6]	validation_0-logloss:0.687051	validation_1-logloss:0.710754
[7]	validation_0-logloss:0.683633	validation_1-logloss:0.711817
[8]	validation_0-logloss:0.683182	validation_1-logloss:0.710874
[9]	validation_0-logloss:0.681139	validation_1-logloss:0.711231
[10]	validation_0-logloss:0.678657	validation_1-logloss:0.721547
[11]	validation_0-logloss:0.676754	validation_1-logloss:0.721922
[12]	validation_0-logloss:0.674994	validation_1-logloss:0.722321


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, early_stopping=50, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=3, missing=None, n_estimators=1000, n_jobs=4,
       nthread=None, objective='reg:linear', random_state=2019,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.85, verbose=1)

In [43]:
import lightgbm as lgb

In [46]:
model1 = lgb.LGBMClassifier(**lgb_params)

In [55]:
model1.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.85,
        importance_type='split', learning_rate=0.05, max_depth=7,
        min_child_samples=46, min_child_weight=0.01, min_split_gain=0.0,
        n_estimators=1000, n_jobs=-1, num_leaves=40, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=2019,
        silent=True, subsample=0.85, subsample_for_bin=200000,
        subsample_freq=2, verbose=-1)